# Parsing Genbank annotations
## Extracting gene boundaries for variant annotation
### August 13th, 2025

This notebook aims to explore parsing Genbank records for variant annotations. Usually a Genbank record represents a single segment with (among other things) annotations of coding regions. There can be multiple genes in one segment, and nucleotides in genes need not be contiguous - splicing can occur, as well as ribosomal slippage sites. These result in the need to correctly parse `CDS` regions, with or without `joins` such as `join(26..51,740..1007)`. This can all be handled by the Biopython library. This work was motivated by the Philadelphia Canine H3N2 sequencing, so we will therefore be exploring a run from the data as of the date of this notebook, as well the `canineh3n2` reference. We will look at the following to ensure comprehensiveness:

- HA ([OR738644.1](https://www.ncbi.nlm.nih.gov/nuccore/OR738644.1/)), single, contiguous gene
- PA ([OR738643.1](https://www.ncbi.nlm.nih.gov/nuccore/OR738643.1/)), gene with a ribosomal slippage site
- PB1 ([OR738642.1](https://www.ncbi.nlm.nih.gov/nuccore/OR738642.1/)), multiple genes on one segment
- MP ([OR738647.1](https://www.ncbi.nlm.nih.gov/nuccore/OR738647.1/)), contains spliced genes

The following function is meant to be a prototype of a "one size" fits all parser for the scenarios described above. The return format is list of dictionaries with the following keys:
- `name` comes from the gene qualifier if available, otherwise "unknown".
- `coding` is the joined nucleotide string (strand-aware).
- `positions` is a list of 0-based genomic positions in the original segment.

In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqFeature import CompoundLocation


def extract_cds_with_coords(genbank_file):
    cds_data = []
    record = SeqIO.read(genbank_file, "genbank")

    for feature in record.features:
        if feature.type != "CDS":
            continue

        seq_parts = []
        coord_parts = []

        # Handle joined locations vs single intervals
        if isinstance(feature.location, CompoundLocation):
            parts = feature.location.parts
        else:
            parts = [feature.location]

        for part in parts:
            start = int(part.start)
            end = int(part.end)
            strand = part.strand

            if strand == 1:
                seq_parts.append(str(record.seq[start:end]))
                coord_parts.extend(range(start, end))
            else:
                seq_parts.append(str(record.seq[start:end].reverse_complement()))
                coord_parts.extend(reversed(range(start, end)))

        coding_seq = "".join(seq_parts)
        assert len(coding_seq) == len(coord_parts)

        gene_name = (
            feature.qualifiers.get("gene", [record.id])[0]
            if "gene" in feature.qualifiers
            else record.id
        )

        cds_data.append({
            "gene": gene_name,
            "coding": coding_seq,
            "positions": coord_parts
        })

    return cds_data


def print_translations(segment):
    gb_filepath = '../data/reference/%s/metadata.gb' % segment
    gb = extract_cds_with_coords(gb_filepath)
    for gene in gb:
        print(gene['gene'], 'gene -', Seq(gene['coding']).translate())

HA compares favorably with the translation in the Genbank record...

In [2]:
print_translations('ha')

HA gene - MKIVIALSYIFCLAFSQNLLGNENNAATLCLGHHAVPNGTMVKTITDDQIEVTNATELVQNSSTGKICNNPHKILDGRDCTLIDALLGDPHCDIFQNETWDLFVERSNAFSNCYPYDVPDYASLRSIIASSGTLEFITEGFTWAGVTQNGGSGACKRGPANSFFSRLNWLTKSGNTYPVLNVTMPNTNNFDKLYIWGVHHPSTDQEQTSLYIQASGRVTVSTRRSQQTIIPNIGSRPLVRGQSGRISVYWTIVKPGDILVINSNGNLIAPRGYFKMHIGKSSIMRSDAPIDTCISECITPNGSIPNEKPFQNVNKITYGACPKYVKQNTLKLATGMRNVPERQTRGLFGAIAGFIENGWEGMVDGWYGFRHQNSEGTGQAADLKSTQAAIDQINGKLNRVIEKTNEKFHQIEKEFSEVEGRIQDLEKYVEDTKIDLWSYNAELLVALENQNTIDLTDSEMNKLFEKTRRQLRENAEDMGNGCFKIYHKCDNACIESIRNGTYDHNIYRDEAVNNRFQIRGVELKSGYKDWILWISFAISCFLLCVVLLGFILWACQRGNIRCNICI*


As does PA for the main gene and the frameshift variant...

In [3]:
print_translations('pa')

PA gene - MEDFVRQCFNPMIVELAEKAMKEYGEDPKIETNKFAAICTHLEVCFMYSDFHFIDERGESIIVEHGDPNALLKHRFEIIEGRDRTMAWTVVNSICNTTEVEKPKFLPDLYDYKENRFIEIGVTRREVHIYYLEKANKIKSEKTHIHIFSFTGEEMATKADYTLDEESRARIKTRLFTIRQEMASRGLWDSFRQSERGEETIEERFEIAGTMRRLADQSLPPNFSSLENFRAYVNGFEPNGYIDGKLSQMSKEVNARIEPFLKTTPRPLRLPDGPPCTQRSKFLLMDALKLSIEDPSHEGEGIPLYDAIKCMKTFFGWKEPNIIKPHKKGINPNYLLAWKQVLAELQDIENEEKIPKTKNMKKTSQLKWVLGENMAPEKVDFEDCKDVGDLKQYDSDEPKPKSLASWIQSEFNKACELTDSSWIELDEIGEDIAPIEHIASKRRNYFTAEVSHCRATEYIMKGVYINTALLNASCAAMDDFQLIPMISKCRTKEGRRKTNLYGFIIKGRSHLRNDTDVVNFVSMEFSLTDPRLEPHKWEKYCVLEIGDMLLRTAIGQVSRPMFLYVRTNGTSKIKMKWGMEMRRCLLQSLQQIESMIEAESSVKEKDMTKEFFENRSETWPIGESPKGVEEGSIGKVCRTLLAKSVFNSLYASPQLEGFSAESRKLLLIVQALRDNLEPGTFDFEGLYEAIEECLINDPWVLLNASWFNSFLTHALK*
PA-X gene - MEDFVRQCFNPMIVELAEKAMKEYGEDPKIETNKFAAICTHLEVCFMYSDFHFIDERGESIIVEHGDPNALLKHRFEIIEGRDRTMAWTVVNSICNTTEVEKPKFLPDLYDYKENRFIEIGVTRREVHIYYLEKANKIKSEKTHIHIFSFTGEEMATKADYTLDEESRARIKTRLFTIRQEMASRGLWDSFVSPKEAKRQLKKGLKSQGPCAGLPTKVSHRTSPALKILEPM*


As do both genes on PB1...

In [4]:
print_translations('pb1')

PB1 gene - MDVNPTLLFLRVPAQNAISTTFPYTGDPPYSHGTGTGYTMDTVNRTHQYSEKGKWTTNTETGAPQLNPIDGPLPEDNEPSGYAQTDCVLEAMAFLEKSHPGIFENSCIETMEVVQQTRVDKLTQGRQTYDWTLNRNQPAATALANTIEVFRSNGLTANESGRLIDFLKDVMESMDKEEMEITTHFQKKRRVRDNMTKKMITQRTIGKKKQRLNKKNYIIRALTLNTMTKDAERGKLKRRAIATPGMQIRGFVYFVETLARSICEKLEQSGLPVGGNEKKAKLANVVRKMMTNSQDTELSFTITGDNTKWNENQNPRMFLAMITYITRNQPEWFRNVLSIAPIMFSNKMARLGKGYMFESKNMKLRTQIPAEMLANINLKYFNESTRKKIEKIRPLLTDGTASLSPGMMMGMFNMLSTVLGVSILNLGQKRYTKSTYWWDGLQSSDDFALIVNAPNHEGIQAGVDRFYRICKLVGINMSKKKSYINRTGTFEFTSFFYRYGFVANFSMELPSFGVSGVNESADMSIGVTVIKNNMINNDLGPATAQMALQLFIKDYRYTYRCHRGDTQIQTRRSFELKKLWEQTRSKAGLLVSDGGPNLYNIRNLHIPEVCLKWELMDEDYQGRLCNPLNPFVSHKEIESVNNAVVMPAHGPAKSMEYDAVTTTHSWIPKRNRSILNTSQRGILEDEQMYQKCCNLFEKFFPSSSYRRPVGISSMVEAMVSRAQIDARIDFESGRIKKEEFAEIMKICSTIEELRRQK*
PB1-F2 gene - MEQGQDTPWTQSIEHINTQRKENGQQTQKLEHPNSIQLMDHYPRIMSQADMHKQIVYWKQWLSLKSPTQGSLKTHVLKRWKLFSKQEWTN*


As do both genes on MP!

In [5]:
print_translations('mp')

M2 gene - MSLLTEVETPTKNEWECKCSDLSDPLIIAASIIGILHLILWILDRLFFKCIYRHLKYGLKRGPSTEGVPESMREEYRQEQQSAVDVDDGHFVNIELE*
M1 gene - MSLLTEVETYVLSIIPSGPLKAEIAQRLEDVFAGKNTDLEALMEWLKTRPILSPLTKGILGFVFTLTVPSERGLQRRRFVQNALNGNGDPNNMDKAVKLYRKLKKEITFHGAKEVALSYSTGALASCMGLIYNRMGTVTTEVAFGLVCATCEQIADSQHRSHRQMVTTTNPLIRHENRMVLASTTAKAMEQMAGSSEQAAEAMEVANQARQMVQAMRTIGTHPSSSAGLKDDLLENLQAYQKRMGVQMQRFK*


These match the respective translations reported in the Genbank files. We now use this to do a refactor of the `extract_coding_regions` and `annotate_amino_acid_changes` functions.

`extract_coding_regions` will now take in a list of all Genbank files, keyed by segment, and return a dictionary, keyed first by segment, then by gene. It is essentially an aggregation of `extract_cds_with_coords` across segments.

In [6]:
segments = ["ha", "mp", "na", "np", "ns", "pa", "pb1", "pb2"]

import json


def extract_coding_regions(segments):
    coding_regions = {}
    for segment in segments:
        genbank_filepath = '../data/reference/%s/metadata.gb' % segment
        coding_regions[segment] = extract_cds_with_coords(genbank_filepath)
    return coding_regions


with open('coding_regions.json', 'w') as f:
    coding_regions = extract_coding_regions(segments)
    json.dump(coding_regions, f)

We now refactor `annotate_amino_acid_changes` to use this new coding regions format. Note that VarScan reports variants in 1-based coordinates, so we must offset.

In [7]:
import csv
import re


def lookup_site(arr, site):
    # return the location of a site in an array, or None if it is not present
    return next(
        (i for i, val in enumerate(arr) if val == site),
        None
    )


amino_acid_abbreviations = {
    "A": "Ala",
    "R": "Arg",
    "N": "Asn",
    "D": "Asp",
    "C": "Cys",
    "Q": "Gln",
    "E": "Glu",
    "G": "Gly",
    "H": "His",
    "I": "Ile",
    "L": "Leu",
    "K": "Lys",
    "M": "Met",
    "F": "Phe",
    "P": "Pro",
    "O": "Pyl",
    "S": "Ser",
    "U": "Sec",
    "T": "Thr",
    "W": "Trp",
    "Y": "Tyr",
    "V": "Val",
    "B": "Asx",
    "Z": "Glx",
    "U": "Sec",
    "X": "Xaa",
    "J": "Xle",
    "*": "Stop",
}


def annotate_amino_acid_changes(coding_regions, vcf, outfilename):
    csvfile = open(vcf, "r")
    outfile = open(outfilename, "w")

    reader = csv.reader(csvfile, delimiter="\t")
    writer = csv.writer(outfile, delimiter="\t")

    header = [
        "segment",
        "gene",
        "reference_position",
        "reference_allele",
        "variant_allele",
        "coding_region_change",
        "synonymous/nonsynonymous",
        "frequency(%)",
        "frequency",
    ]
    writer.writerow(header)

    for row in reader:
        # ignore comment lines
        if "##" in row[0] or "#CHROM" in row[0]:
            continue

        segment = row[0]
        site = int(row[1]) - 1  # to make this 0 indexed
        reference_allele = row[3].lower()
        alternative_allele = row[4].lower()
        
        # pull out the frequency using a string search
        SearchStr = r".+\:([0-9]{1,2}\.{0,1}[0-9]{0,2}\%)"
        result = re.search(SearchStr, row[9])
        if result:
            frequency = "\t".join(result.groups())
            frequency_decimal = frequency.replace("%", "")
            frequency_decimal = (float(frequency_decimal)) / 100
            frequency_decimal = (
                "%.4f" % frequency_decimal
            )  # only include 4 numbers after decimal
        else:
            frequency = "none reported"

        for gene in coding_regions[segment]:
            gene_name = gene['gene']
            cds = gene['coding']
            positions = gene['positions']
            position = lookup_site(positions, site)
            if position is None:
                continue
            # how far back should we go for the start of the codon?
            # zero-based indexing and modular arithmetic pair well here
            offset = position % 3
            start = position - offset
            stop = start + 3
            ref_codon = cds[start: stop]
            ref_aa = Seq(ref_codon).translate()

            # we convert to a list to dynamically modify a base, then join to a string
            variant_codon_list = list(ref_codon)
            variant_codon_list[offset] = alternative_allele
            variant_codon = ''.join(variant_codon_list)
            variant_aa = Seq(variant_codon).translate()

            # return the amino acid changes, with single letters converted to 3-letter aa codes
            if ref_aa == variant_aa:
                syn_nonsyn = "synonymous"
            elif ref_aa != variant_aa and variant_aa == "*":
                syn_nonsyn = "stop_gained"
            elif ref_aa != variant_aa:
                syn_nonsyn = "nonsynonymous"
                            
            # now determine whether the site is in the 1st, 2nd, or 3rd codon position
            # if SNP is in 1st position in codon:
            aa_site = int(position / 3) + 1
            
            amino_acid_change = (
                amino_acid_abbreviations[ref_aa]
                + str(aa_site)
                + amino_acid_abbreviations[variant_aa]
            )
            output = [
                segment,
                gene_name,
                str(site + 1),
                reference_allele.upper(),
                alternative_allele.upper(),
                amino_acid_change,
                syn_nonsyn,
                frequency,
                frequency_decimal
            ]
            writer.writerow(output)

    csvfile.close()
    outfile.close()
    return

annotate_amino_acid_changes(
    coding_regions,
    '../data/c2/replicate-1/remapping-2/varscan.vcf',
    'varscan-annotated.tsv'
)

This runs! Let's compare this implementation with a previous one that ran. This implementation should be more robust to:
- variants that appear in the first exon for a gene with 2+ exons
- variants that appear at an intron/exon boundary
- frameshift variants

In [8]:
import pandas as pd


pd.read_csv('../data/c13/replicate-2/remapping-2/varscan-annotated.tsv', sep='\t')

,segment,gene,reference_position,reference_allele,variant_allele,coding_region_change,synonymous/nonsynonymous,frequency(%),frequency,Unnamed: 9
0,pb2,PB2,303,T,C,Ser92Ser,synonymous,1.5%,0.0150,NaN
1,pb2,PB2,395,A,G,Glu123Gly,nonsynonymous,1.74%,0.0174,NaN
2,pb2,PB2,498,A,G,Lys157Lys,synonymous,1.52%,0.0152,NaN
3,pb2,PB2,546,A,G,Gly173Gly,synonymous,2.4%,0.0240,NaN
4,pb2,PB2,675,G,A,Arg216Arg,synonymous,4.19%,0.0419,NaN
...,...,...,...,...,...,...,...,...,...,...
181,ns,NS1,568,T,C,Leu181Pro,nonsynonymous,2.86%,0.0286,NaN
182,ns,NEP,668,C,T,Ser57Phe,nonsynonymous,3.01%,0.0301,NaN
183,ns,NS1,668,C,T,Leu214Leu,synonymous,3.01%,0.0301,NaN
184,ns,NEP,724,A,G,Ile76Val,nonsynonymous,1.45%,0.0145,NaN


In [9]:
annotate_amino_acid_changes(
    coding_regions,
    '../data/c13/replicate-2/remapping-2/varscan.vcf',
    'c13-rep2-varscan-annotated.tsv'
)
new = pd.read_csv('c13-rep2-varscan-annotated.tsv', sep='\t')
new

,segment,gene,reference_position,reference_allele,variant_allele,coding_region_change,synonymous/nonsynonymous,frequency(%),frequency
0,pb2,PB2,303,T,C,Ser92Ser,synonymous,1.5%,0.0150
1,pb2,PB2,395,A,G,Glu123Gly,nonsynonymous,1.74%,0.0174
2,pb2,PB2,498,A,G,Lys157Lys,synonymous,1.52%,0.0152
3,pb2,PB2,546,A,G,Gly173Gly,synonymous,2.4%,0.0240
4,pb2,PB2,675,G,A,Arg216Arg,synonymous,4.19%,0.0419
...,...,...,...,...,...,...,...,...,...
181,ns,NS1,568,T,C,Leu181Pro,nonsynonymous,2.86%,0.0286
182,ns,NEP,668,C,T,Ser57Phe,nonsynonymous,3.01%,0.0301
183,ns,NS1,668,C,T,Leu214Leu,synonymous,3.01%,0.0301
184,ns,NEP,724,A,G,Ile76Val,nonsynonymous,1.45%,0.0145


We observe broad agreement between the tools on the simplest cases. Most notably is that there is agreement on `PA-X` with the new tool up until the slippage site at 594, after which we observe expected disagreement. To further confirm, let's look at amino acids in the translation of PA-X and ensure they compare with the calls. 

In [10]:
pa_x = new.loc[new.gene == 'PA-X', 'coding_region_change']
pa_x

58      Glu2Gly
60     Leu42Leu
62     Glu59Glu
64     Lys73Arg
66    Phe105Ser
68    Lys134Glu
70    Lys142Arg
72    Phe148Ser
74    Ser193Ser
76    Lys202Arg
78    Glu230Gly
Name: coding_region_change, dtype: object

In [11]:
positions = pa_x.apply(
    lambda x: int(x[3:-3])
)
positions

58      2
60     42
62     59
64     73
66    105
68    134
70    142
72    148
74    193
76    202
78    230
Name: coding_region_change, dtype: int64

In [12]:
ref = pa_x.apply(
    lambda x: x[:3]
)
ref

58    Glu
60    Leu
62    Glu
64    Lys
66    Phe
68    Lys
70    Lys
72    Phe
74    Ser
76    Lys
78    Glu
Name: coding_region_change, dtype: object

In [13]:
aa = Seq(coding_regions['pa'][1]['coding']).translate()
[amino_acid_abbreviations[aa[i-1]] for i in positions]

['Glu', 'Leu', 'Glu', 'Lys', 'Phe', 'Lys', 'Lys', 'Phe', 'Ser', 'Lys', 'Glu']

This provides a more robust and broadly applicable tool for annotating variants in viral deep sequencing data.